In [4]:
from lxml import etree
import base64
import sqlite3
import pandas as pd
import cairosvg
from PyPDF2 import PdfMerger
from datetime import datetime, timedelta
from io import BytesIO
import os
from modules.data_utils import fetch_agent_contact_info

# Define namespaces
NSMAP = {
    None: "http://www.w3.org/2000/svg",
    "xlink": "http://www.w3.org/1999/xlink",
    "inkscape": "http://www.inkscape.org/namespaces/inkscape",
    "sodipodi": "http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd"
}

def load_svg(input_svg_path):
    """
    Load and parse an SVG file.

    Args:
        input_svg_path (str): Path to the input SVG file.

    Returns:
        tuple: A tuple containing the parsed tree and root element, or (None, None) if an error occurs.
    """
    try:
        tree = etree.parse(input_svg_path)
        root = tree.getroot()
        return tree, root
    except FileNotFoundError:
        print(f"Error: SVG file not found at '{input_svg_path}'")
    except etree.XMLSyntaxError as e:
        print(f"Error parsing SVG file: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None, None

def replace_text(root, element_id, old_text, new_text, namespaces):
    text_element = root.find(f".//*[@id='{element_id}']", namespaces=namespaces)
    if text_element is not None:
        tspan_x = text_element.get("x")
        tspan_y = text_element.get("y")
        updated_text = text_element.text.replace(old_text, str(new_text))
        text_element.text = updated_text
        if tspan_x is not None:
            text_element.set("x", tspan_x)
        if tspan_y is not None:
            text_element.set("y", tspan_y)
        # Ensure parent <tspan> or <text> doesn’t override
        parent = text_element.getparent()
        if parent.tag == "{http://www.w3.org/2000/svg}tspan":
            parent_x = parent.get("x")
            parent_y = parent.get("y")
            if parent_x is not None:
                parent.set("x", parent_x)
            if parent_y is not None:
                parent.set("y", parent_y)
        #print(f"Stabilized <tspan> at x={tspan_x}, y={tspan_y}")

def embed_svg(root, base_svg_path, embed_svg_path, x=0, y=0, scale=1.0):
    """
    Embed an SVG file into another SVG file at a specified position and scale, modifying the base SVG in-place.
    
    Args:
        root: The root element of the base SVG tree to modify
        base_svg_path: Path to the base SVG file
        embed_svg_path: Path to the SVG file to embed
        x: X-coordinate for the embedded SVG's top-left corner (default: 0)
        y: Y-coordinate for the embedded SVG's top-left corner (default: 0)
        scale: Scaling factor for the embedded SVG (default: 1.0)
    
    Returns:
        The modified SVG tree
    """
    try:
        # Parse the base SVG
        base_tree = root.getroottree()

        # Parse the SVG to embed
        embed_tree = etree.parse(embed_svg_path)
        embed_root = embed_tree.getroot()

        # Create a <g> element to group the embedded SVG content
        group = etree.Element("g", nsmap=NSMAP)
        transform = f"translate({x}, {y}) scale({scale})"
        group.set("transform", transform)

        # Move all children from the embed SVG's root to the group
        for child in embed_root:
            group.append(child)

        # Append the group to the base SVG's root
        root.append(group)

        # Save the modified SVG
        print(f"SVG embedded successfully")
        return base_tree

    except etree.XMLSyntaxError as e:
        print(f"Error parsing embedded SVG: {e}")
    except FileNotFoundError as e:
        print(f"Error: Embedded SVG file not found - {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None


2025-04-16 21:35:17.221 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 21:35:17.225 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 21:35:17.230 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 21:35:17.232 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-16 21:35:17.236 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [ ]:
def process_page7(IN, media_mensal, total_contrato, economia_contratual, economia_anual, input_svg_path="Proposta PPT/page 7.svg", output_svg_path="Temp_ppt/page 7.svg", db_path="DataBase.db"):
    
    validade = datetime.today() + timedelta(days=5)

    # Load the SVG file
    tree, root = load_svg(input_svg_path)
    if not tree or not root:
        return
    
    # Replace text in the SVG file
    replace_text(root, "tspan520-74-4-0-6", "XXXXXXXXXX", IN, NSMAP) 
    replace_text(root, "tspan520-7", "xx xxx,xx", f"{media_mensal:,.2f}".replace(",", " ").replace(".",","), NSMAP) 
    replace_text(root, "tspan520-7-1", "xx xxx,xx", f"{total_contrato:,.2f}".replace(",", " ").replace(".",","), NSMAP)
    replace_text(root, "tspan5", "XXX XXX", f"{total_contrato:,.0f}".replace(",", " ").replace(".",","), NSMAP)
    replace_text(root, "tspan1", "25%",  f"{economia_contratual:.0%}" , NSMAP)    
    #replace_text(root, "tspan2","14%", f"{economia_efetiva:.0%}" , NSMAP)
    replace_text(root, "tspan12","20/02/2025", f"{validade.day:02d}/{validade.month:02d}/{validade.year}", NSMAP)  
    embed_svg(root, input_svg_path,"images/energy_cost_plot.svg" ,x=125,y=75, scale= 0.18)
    embed_svg(root, input_svg_path,"images/historic_graph.svg" ,x=80,y=170, scale= 0.165)

    # Save the modified SVG file
    try:
        tree.write(output_svg_path, pretty_print=True, xml_declaration=True, encoding="utf-8")
        print(f"Modified SVG saved to '{output_svg_path}'")

    except IOError as e:
        print(f"Error saving modified SVG: {e}")
    
    except Exception as e:
        print(f"Unexpected error: {e}")

process_page7("30411253", 4370.00, 157370.00, 0.26, 0.12)

SVG embedded successfully
SVG embedded successfully
Modified SVG saved to 'Temp_ppt/page 7.svg'


C:\Users\Rodri\AppData\Local\Temp\ipykernel_37336\3065713834.py:15: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not tree or not root:


In [1]:
from modules.plot_generator import create_historic_graph

In [9]:

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
vs_cativo = [620, 610, 630, 600, 640, 650, 660, 670, 680, 690, 700, 710]
v_desconto = [12, 14, 13, 15, 14, 13, 12, 14, 13, 15, 14, 13]
vs_fio_acl = [0.75 * cativo - desconto for cativo, desconto in zip(vs_cativo, v_desconto)]

create_historic_graph(months, vs_cativo, v_desconto, vs_fio_acl, figsize=(16,5))